### Encoder와 Decoder 구현

![img1](./img/img1.png)

### Encoder 세부 구조

* selfattention 레이어 끼리는 서로 연관성이 있지만 feed forward 사이에는 개별성이 있음. 
* feed forward는 개별성이 있기 때문에 parallel이 가능한 구조가 됨.

![img3](./img/img3.png)

### Multi_Headed_Attention

* Multi_Headed_Attention = Heads * SelfAttention

* embeding_size = Token 개수 | parameter

* Heads = Attention 개수 | parameter

* Attention_dim = embeding_size // Heads



### Self Attention 구현
* **단어와의 연관성, 중요도를 계산하는 레이어**

    ![img4](./img/img4.png)

* embed_size : 토큰 개수 , 논문에서는 512개로 사용

    ![img2](./img/img2.png)

### Query, Key, Value 

* self attention이 단어와의 연관성을 파악하기 위해서 query, key, value라는 개념을 사용한다.
    - query : 현재 관심을 갖는 단어 단어 | vector
    - key : column 명이라고 보면 될듯 vector | vector
    - value : column 안에 있는 실제 값인 듯 vector | vector 

    ![img5](./img/img5.png)

* Score = query와 key의 곱을 한뒤 attention 차원의 루트 값으로 나눈다.
    
    $scaled dot product attention = score(p,k) = \frac{Q K^\text{T}}{\sqrt{d}}$

* Back Propagation을 통해 query, key, value의 W을 찾아야한다.
    - forward Propagation 예시 : $embedding(1*4) * W^Q (4*3) = q_1(1*3)$
    - 위 식을 통해서 back propagation을 구현해서 만든다. 
    - 

    ![img6](./img/img6.png)

    - query, key, value는 각각 64차원 * 8(multi-headed-attention 개수 ) = embed_size
    - head가 8개라는건 양옆 4개 단어를 비교하겠다는건가..?

### query, key, value W를 계산하는 절차

* Step 1: Create three vectors from each of the encoder’s input vectors

    `head_dim = embed_size // heads`

* Step 2: Calculate a score

    * 처음에는 W값들이 임의로 구해지나?


* Step 3: Divide the score by  $\sqrt{b_k}$
    이렇게 해야 성능이 잘나온다고 논문에 나옴.

* Step 4: Pass the result through a softmax operation
    지금 보고 있는 토큰과 얼마나 관련성이 있는지 확률로 변환

* Step 5: Multiply each value vector by the softmax score
    v에다가 퍼센트를 곱해서 단어의 중요도를 반영

* Step 6 : Sum up the weighted value vector which produces the output of the self- attention layer at this position
    Step 5에서 얻은 값들을 모두 더하면 self attention 값이 된다.


단계 한눈에 보기

![img7](./img/img7.png)

### 계산 절차를 병렬로 계산하기 (Matrix calulation of self attention)

$\begin{equation}
 Attention(Q,K,V)= Z = \mathrm{softmax} \left( \frac{Q K^\text{T}}{\sqrt{d}} \right) V
\end{equation}$

![img8](./img/img8.png)

### Multi-headed attention

* attention($z_n$) 결과 값을 모두 concat($z_1 + z_2 + ... + z_8 $) = $z_{all}$
* $ z_{all} * W^0 = Z $

    ![img9](./img/img9.png)

### Z는 Feed Forward에 투입되는 값이다!

In [2]:
import torch
import torch.nn as nn

class selfAttention(nn.Module) :
    def __init__(self,embed_size, heads) -> None:
        '''
        embed_size : input 토큰 개수, 논문에서는 512개로 사용 
        heads : multi_head의 개수, 논문에서는 8개 사용
        
        '''
        # nn.Module의 parameter 불러오기
        super(selfAttention,self).__init__() 
        # 왜 selfAttention을 또 불러오는지 모르겠네..
        # 성능상 이슈는 아니고 참고용으로 사용할때 도움되는거라고 함.

        
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size //heads
        
        # Linear 함수느 어떤 용도로 쓰이는거지?
        '''
        values :
        keys :
        queries : 
        '''
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads*self.head_dim, embed_size)
        
    def forward(self, values,keys,query,mask) :
        N = query.shape[0] 
        # tensor shape
        value_len = values.shape[1] 
        key_len = keys.shape[1] 
        query_len = query.shape[1]

        values = values.reshape(N,value_len, self.heads,self.head_dim)
        keys = keys.reshape(N,key_len, self.heads,self.head_dim)
        queries = queries.reshape(N,query_len, self.heads,self.head_dim)

        # score 설명 참고
        # einsum이 뭐지
        score = torch.einsum("nqhd,nkhd->nhqk", [queries,keys]) 
        # queries shape : N,value_len, self.heads,self.head_dim
        # keys shape : N,key_len, self.heads,self.head_dim
        # score shape : N, heads, query_len, key_len
        
        # decoder 구조인 masked Self Attention 적용 시 활용되는 구문
        # decoder 설명시 상세 소개
        if mask is not None :
            score = score.masked_fill(mask == 0, float("-1e20"))
            '''
            -1e20 = -inf
            -inf이기 때문에 값이 0에 수렴(0은 안됨)
            mask가 부여된 경우 score 값을 0으로 준다.

            '''

        attention = torch.softmax(score / self.embed_size**(1/2),dim=3)

        out = torch.einsum("nhql,nlhd -> nqhd",[attention, values]).reshape(
            N,query_len,self.heads,self.head_dim
            )
        # attention shape : N, heads,query_len,key_len
        # values shape : N, value_len, heads, heads_dim
        # out shape : N, query_len, heads, head_dim

        # concat all heads 
        out = self.fc_out(out)

        return out
        

### Encoder Block 구현하기

### Add(=residual connection) & Norm
* add(=residual connection)를 하는 이유는 gradient descent 할 때 0이 될 수 있기 때문
* 방법 ; z 에다가 x를 더한다. 이때 x 는 이전 z 값 z는 현재 z값



### LayerNorm

### PointWize Feed Forward
* 같은 계층의 encoder에서 feed forward 구조가 같음
* Linear(512d)->relu(2048d) -> Linear(512d)

![img10](./img/img10.png)

In [3]:
class transformerBlock(nn.Module) :
    def __init__(self,embed_size, heads, dropout, forward_expansion) -> None:
        '''
        embed_size : token 개수 | 논문 512개
        heads : attention 개수
        dropout :
        forward_expansion : forward 계산시 차원을 얼마나 늘릴 것인지 결정, 임의로 결정하는 값
                            forward_차원 계산은 forward_expension * embed_size 
                            논문에서는 4로 정함. 총 2048차원으로 늘어남.

        '''
        super().__init__()
        self.attention = selfAttention(embed_size,heads)
        
        # LayerNorm 장점 및 적용하는 이유는?
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forawrd = nn.Sequential(
            # 차원을 512 -> 2048로 증가
            nn.Linear(embed_size,forward_expansion*embed_size),
            # 차원을 Relu 연산
            nn.ReLU(),
            # 차원 2048 -> 512로 축소 
            nn.Linear(forward_expansion*embed_size,embed_size)
            )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value,key,query,mask) :
        # 왜 attention.forward가 아니고 그냥 쓰는거지?
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))

        forward = self.feed_forawrd(x)
        out = self.dropout(self.norm2(forward + x))
        return out 